In [1]:
import pandas as pd
import numpy as np
import spacy
import textblob
from gensim.models import word2vec
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels import robust
from string import punctuation
import gensim
from gensim.models import word2vec

import warnings
warnings.filterwarnings('ignore')

from sklearn.cluster import SpectralClustering
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Neural Network
from sklearn.neural_network import MLPClassifier

import keras
from keras.optimizers import RMSprop,SGD
from keras.layers import Conv1D,Conv2D, MaxPooling2D,GlobalMaxPooling1D
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.layers import LSTM, Embedding

Using TensorFlow backend.


In [2]:
from dask.distributed import Client, progress
import dask.dataframe as dd
import dask.array as da

In [3]:
import os
# for mac only: frog,blow,funk,glass,tink,submarine,purr,sosumi
def beep(audio): 
    os.system('afplay /System/Library/Sounds/' + audio +'.aiff')

## Create Dask Client

In [26]:
from dask.distributed import Client, progress
client = Client(n_workers=4, threads_per_worker=1, memory_limit='2GB')
client

Client Scheduler: tcp://127.0.0.1:50001 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.00 GB


In [28]:
review.head()

,Compiled from boardgamegeek.com by Matt Borthwick,gameID,rating,comment
228,103046,178900,9.0,Really nice. Works best for me when people dea...
636,71939,121921,8.5,"I love games that tell stories, and this one c..."
630,121536,43111,5.0,"Wargame, domination, king-of-the-mountain type..."
17,29353,181,4.0,This game used to be fun when I played it as a...
645,20995,478,7.0,I have actaully played this many more times th...


tornado.application - ERROR - Exception in callback <bound method SystemMonitor.update of <SystemMonitor: cpu: 21 memory: 61 MB fds: 146>>
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/tornado/ioloop.py", line 1208, in _run
    return self.callback()
  File "/anaconda3/lib/python3.6/site-packages/distributed/system_monitor.py", line 71, in update
    read_bytes = (ioc.bytes_recv - last.bytes_recv) / (duration or 0.5)
AttributeError: 'NoneType' object has no attribute 'bytes_recv'
Exception ignored in: <generator object add_client at 0x1c2e5bb0a0>
RuntimeError: generator ignored GeneratorExit
Future exception was never retrieved
future: <Future finished exception=CommClosedError('in <closed TCP>: Stream is closed',)>
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/distributed/comm/tcp.py", line 179, in read
    n_frames = yield stream.read_bytes(8)
  File "/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1

## Word Embedding - Word2Vec

In [27]:
review = dd.read_csv('boardgame-comments-sample.csv').sample(frac=.1,random_state=42)
review = client.persist(review)

In [6]:
review.columns = 'reviewer_id', 'game_id', 'rating', 'comment'

# RATINGS ADJUSTMENT: ceiling >= .5 [or] floor < .5
ratings = da.array(review.rating.round()).compute()

print('Total Comments: {}'.format(len(review.comment)))

Total Comments: 85


In [7]:
%%time
sentences = da.array(review.comment.apply(lambda val: [b.lower() for b in textblob.TextBlob(val).words])).compute()
beep('ping')

CPU times: user 130 ms, sys: 20.2 ms, total: 150 ms
Wall time: 1.71 s


In [8]:
%%time
load_model = False

if load_model:
    # # load model
    word_vec = word2vec.Word2Vec.load('full_word2vec_blob.bin')
    vec_size = word_vec.layer1_size
else: 
    vec_size = 50
    word_vec = word2vec.Word2Vec(
        sentences,
        workers=4,     # Number of threads to run in parallel (if your computer does parallel processing).
        min_count=5,  # Minimum word count threshold.
        window=6,      # Number of words around target word to consider.
        sg=0,          # Use CBOW because our corpus is small.
        sample=1e-3 ,  # Penalize frequent words.
        size=vec_size,      # Word vector length.
        hs=1           # Use hierarchical softmax.
    )
    
    # save model
    word_vec.save('full_word2vec_blob.bin')

# List of words in model.
vocab = word_vec.wv.vocab.keys()
beep('ping')

CPU times: user 97.2 ms, sys: 13 ms, total: 110 ms
Wall time: 923 ms


In [9]:
%%time
vec_new = np.array([.5 for i in range(0,vec_size)])
vectors = da.array(map(lambda val: [word_vec[w] if w in vocab else vec_new for w in val], sentences))
beep('ping')

CPU times: user 48.3 ms, sys: 7.01 ms, total: 55.3 ms
Wall time: 853 ms


### TBD

# 6. Comprehensive Neural Network

In [12]:
%%time
pad, max_words = np.array([0 for i in range(0,vec_size)]), 30

def manual_pad(val):
    empty = max_words-len(val)
    for i in range(0,empty):
        val.append(pad)
    
    return [i for i in val[0:max_words]]

vectors_padding = list(map(manual_pad,vectors))
beep('ping')

CPU times: user 95.9 ms, sys: 11.7 ms, total: 108 ms
Wall time: 926 ms


In [13]:
client.close()

Future exception was never retrieved
future: <Future finished exception=CommClosedError('in <closed TCP>: Stream is closed',)>
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/distributed/comm/tcp.py", line 179, in read
    n_frames = yield stream.read_bytes(8)
  File "/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1099, in run
    value = future.result()
tornado.iostream.StreamClosedError: Stream is closed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1107, in run
    yielded = self.gen.throw(*exc_info)
  File "/anaconda3/lib/python3.6/site-packages/distributed/comm/tcp.py", line 200, in read
    convert_stream_closed_error(self, e)
  File "/anaconda3/lib/python3.6/site-packages/distributed/comm/tcp.py", line 128, in convert_stream_closed_error
    raise CommClosedError("in %s: %s" % (obj, exc))
distributed.comm.

In [14]:
%%time
y = np.array(ratings).astype(int).ravel()
X = np.array(vectors_padding)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

CPU times: user 2.74 ms, sys: 1.24 ms, total: 3.99 ms
Wall time: 4.92 ms


AttributeError: 'Future' object has no attribute 'head'

In [15]:
XtrSize,XtrMax,XtrVector = X_train.shape
XteSize,XteMax,XteVector = X_test.shape

#### _A - Logistic Regression w/ Vectors_

#### _B - MLP Neural NN w/ Vectors_

#### _C - Keras Sequential NN_

In [16]:
y_test = y_test.ravel()
y_train = y_train.ravel()

y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [17]:
hidden_size = 500
batch_size = 4
epochs = 1

In [18]:
X_train.shape

(56, 30, 50)

array([[-0.03219954, -0.03153344,  0.02508014, ..., -0.01869514,
         0.02046979,  0.05593041],
       [-0.05245737, -0.06604368,  0.0318367 , ..., -0.0615892 ,
         0.04242897,  0.13361858],
       [-0.0875781 , -0.10805944,  0.07053864, ..., -0.09883013,
         0.05956834,  0.20717934],
       ...,
       [-0.03251036, -0.03827142,  0.02689563, ..., -0.03629916,
         0.0072948 ,  0.0600408 ],
       [-0.05266681, -0.05059045,  0.04314951, ..., -0.04447406,
         0.02799511,  0.10280135],
       [-0.06484579, -0.08580186,  0.04270741, ..., -0.06139933,
         0.04014385,  0.14744286]])

In [20]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=X.shape[1:]))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Conv2D(2,2,padding='valid', activation='relu',strides=2))
model.add(GlobalMaxPooling1D())
model.add(Dense(11, activation='softmax'))

In [21]:
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(),
              metrics=['accuracy'])

In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 30, 512)           26112     
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 512)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 30, 512)           262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 30, 512)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 30, 33)            16929     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 33)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 11)                374       
Total para

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_test, y_test))

beep('ping')

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## Word Similarity Visualization